import sys

from PyQt5.QtWidgets import *

from PyQt5.QAxContainer import *

from PyQt5.QtCore import *

In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
from PyQt5.QtCore import *

class Kiwoom(QAxWidget):
    def __init__(self):
        super().__init__()
        self._create_kiwoom_instance()
        self._set_signal_slots()

    def _create_kiwoom_instance(self):
        self.setControl("KHOPENAPI.KHOpenAPICtrl.1")

    def _set_signal_slots(self):
        self.OnEventConnect.connect(self._event_connect)

    def comm_connect(self):
        self.dynamicCall("CommConnect()")
        self.login_event_loop = QEventLoop()
        self.login_event_loop.exec_()

    def _event_connect(self, err_code):
        if err_code == 0:
            print("connected")
        else:
            print("disconnected")

        self.login_event_loop.exit()

    def get_code_list_by_market(self, market):
        code_list = self.dynamicCall("GetCodeListByMarket(QString)", market)
        code_list = code_list.split(';')
        return code_list[:-1]

if __name__ == "__main__":
    app = QApplication(sys.argv)
    kiwoom = Kiwoom()
    kiwoom.comm_connect()
    code_list = kiwoom.get_code_list_by_market('10')
    for code in code_list:
        print(code, end=" ")


connected
900110 900270 900260 900290 900300 900310 900340 000250 000440 001000 001540 001810 001840 002230 002290 002680 002800 003100 003310 003380 003800 004590 004650 004780 005160 005290 005670 005710 005860 005990 006050 006140 006580 006620 006730 006910 006920 007330 007370 007390 007530 007680 007720 007770 007820 008290 008370 008470 008830 009300 009520 009620 009730 009780 010170 010240 010280 010470 011040 011080 011320 011370 011560 012340 012620 012700 012790 012860 013030 013120 013310 013720 013810 013990 014100 014190 014200 014470 014570 014620 014940 014970 015710 015750 016100 016250 016600 016670 016790 016920 017000 017250 017480 017510 017650 017890 018000 018120 018290 018310 018620 018680 018700 019010 019210 019540 019550 019570 019590 019660 019770 019990 020180 020400 020710 021040 021045 021080 021320 021650 021880 022100 022220 023160 023410 023440 023460 023600 023760 023770 023790 023900 023910 024060 024120 024740 024800 024810 024830 024840 024850 024

In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
from PyQt5.QtCore import *
import time

TR_REQ_TIME_INTERVAL = 0.2

class Kiwoom(QAxWidget):
    def __init__(self):
        super().__init__()
        self._create_kiwoom_instance()
        self._set_signal_slots()

    def _create_kiwoom_instance(self):
        self.setControl("KHOPENAPI.KHOpenAPICtrl.1")

    def _set_signal_slots(self):
        self.OnEventConnect.connect(self._event_connect)
        self.OnReceiveTrData.connect(self._receive_tr_data)

    def comm_connect(self):
        self.dynamicCall("CommConnect()")
        self.login_event_loop = QEventLoop()
        self.login_event_loop.exec_()

    def _event_connect(self, err_code):
        if err_code == 0:
            print("connected")
        else:
            print("disconnected")

        self.login_event_loop.exit()

    def get_code_list_by_market(self, market):
        code_list = self.dynamicCall("GetCodeListByMarket(QString)", market)
        code_list = code_list.split(';')
        return code_list[:-1]

    def get_master_code_name(self, code):
        code_name = self.dynamicCall("GetMasterCodeName(QString)", code)
        return code_name

    def set_input_value(self, id, value):
        self.dynamicCall("SetInputValue(QString, QString)", id, value)

    def comm_rq_data(self, rqname, trcode, next, screen_no):
        self.dynamicCall("CommRqData(QString, QString, int, QString", rqname, trcode, next, screen_no)
        self.tr_event_loop = QEventLoop()
        self.tr_event_loop.exec_()

    def _comm_get_data(self, code, real_type, field_name, index, item_name):
        ret = self.dynamicCall("CommGetData(QString, QString, QString, int, QString", code,
                               real_type, field_name, index, item_name)
        return ret.strip()

    def _get_repeat_cnt(self, trcode, rqname):
        ret = self.dynamicCall("GetRepeatCnt(QString, QString)", trcode, rqname)
        return ret

    def _receive_tr_data(self, screen_no, rqname, trcode, record_name, next, unused1, unused2, unused3, unused4):
        if next == '2':
            self.remained_data = True
        else:
            self.remained_data = False

        if rqname == "opt10081_req":
            self._opt10081(rqname, trcode)

        try:
            self.tr_event_loop.exit()
        except AttributeError:
            pass

    def _opt10081(self, rqname, trcode):
        data_cnt = self._get_repeat_cnt(trcode, rqname)

        for i in range(data_cnt):
            date = self._comm_get_data(trcode, "", rqname, i, "일자")
            open = self._comm_get_data(trcode, "", rqname, i, "시가")
            high = self._comm_get_data(trcode, "", rqname, i, "고가")
            low = self._comm_get_data(trcode, "", rqname, i, "저가")
            close = self._comm_get_data(trcode, "", rqname, i, "현재가")
            volume = self._comm_get_data(trcode, "", rqname, i, "거래량")
            print(date, open, high, low, close, volume)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    kiwoom = Kiwoom()
    kiwoom.comm_connect()

    # opt10081 TR 요청
    kiwoom.set_input_value("종목코드", "039490")
    kiwoom.set_input_value("기준일자", "20170224")
    kiwoom.set_input_value("수정주가구분", 1)
    kiwoom.comm_rq_data("opt10081_req", "opt10081", 0, "0101")

    while kiwoom.remained_data == True:
        time.sleep(TR_REQ_TIME_INTERVAL)
        kiwoom.set_input_value("종목코드", "039490")
        kiwoom.set_input_value("기준일자", "20170224")
        kiwoom.set_input_value("수정주가구분", 1)
        kiwoom.comm_rq_data("opt10081_req", "opt10081", 2, "0101")


connected
20170224 76900 77900 76000 76300 57565
20170223 78100 78100 76000 77000 60117
20170222 76600 78100 76300 77300 49600
20170221 73500 76900 73300 76500 46682
20170220 74600 75000 73600 73800 30275
20170217 74600 75900 73900 75600 30195
20170216 72100 75900 72100 75800 67704
20170215 71700 73600 71400 71800 72560
20170214 73700 73700 70900 71700 65403
20170213 72300 74000 71300 73700 74018
20170210 71500 72900 70100 72300 98638
20170209 71400 71600 70800 71300 66026
20170208 72900 72900 70900 71500 44520
20170207 74200 74300 72300 72800 43150
20170206 71200 74600 70800 74600 85693
20170203 70900 71400 69800 70300 34804
20170202 72800 72800 70900 70900 51492
20170201 72500 72600 71900 72600 39271
20170131 72200 72800 71300 72600 35330
20170126 72000 74900 71500 73800 64048
20170125 72000 72300 71100 72000 18214
20170124 73000 73000 70700 71900 52194
20170123 72500 73400 72200 73000 10598
20170120 74500 74500 72000 72100 66516
20170119 74200 76300 74200 74500 65013
20170118 74700 

20151130 60400 60400 57100 58600 40747
20151127 60600 60700 59200 60300 34082
20151126 60500 61000 59300 60300 40920
20151125 60300 60800 60200 60400 37438
20151124 59000 60800 58700 60300 46953
20151123 59500 60300 58300 59000 22884
20151120 60300 60300 59100 59500 31947
20151119 58400 59800 57900 59800 61806
20151118 56800 58400 56800 58300 49565
20151117 54700 58000 54700 57300 74048
20151116 53500 54900 52900 54600 50637
20151113 53700 54200 53300 53800 40981
20151112 54400 54500 53800 54100 24352
20151111 54100 55300 53400 54800 27736
20151110 56000 56100 53800 54600 37452
20151109 58300 58500 55700 56000 29955
20151106 57900 59000 57400 58500 27472
20151105 57800 58000 57100 57900 19287
20151104 57500 58100 56800 57500 24707
20151103 57200 58200 56900 57900 36451
20151102 57600 58700 56900 57200 31924
20151030 58600 58700 57600 58700 38702
20151029 59100 59200 57300 58200 22974
20151028 58100 59300 58100 58400 37744
20151027 57400 59300 57400 58300 39904
20151026 58100 58200 5760

20141029 45400 45600 45000 45300 71778
20141028 44700 45050 44350 44900 17894
20141027 44200 45200 44200 44850 20365
20141024 44750 44950 44300 44300 14797
20141023 45250 45850 44300 44500 30264
20141022 45850 45900 44900 45600 31327
20141021 45000 45650 44600 45000 45682
20141020 44550 45050 44300 44900 33252
20141017 43850 44200 43450 44000 44209
20141016 45200 45650 43600 43650 114469
20141015 46550 46700 45400 45700 37624
20141014 45800 46950 45600 46500 87766
20141013 46750 47000 45400 46400 35488
20141010 46800 47700 45350 46900 39180
20141008 47550 48250 47050 47100 22107
20141007 47300 48450 47300 48050 33470
20141006 48700 49050 47550 47550 41455
20141002 47750 48600 46400 48550 77514
20141001 51300 51300 48000 48200 78946
20140930 50200 51300 49600 51200 55935
20140929 52000 52000 50400 51000 57932
20140926 50100 52100 50000 51600 52887
20140925 52200 52300 50800 51100 25527
20140924 52000 52000 51200 51500 22839
20140923 51500 52400 50800 51900 32737
20140922 52000 52500 515

20130628 56000 56500 55000 55300 40993
20130627 54500 56600 54500 56200 64460
20130626 55100 55100 53300 54100 37675
20130625 55000 55600 53700 53700 51893
20130624 56600 56600 55500 55800 30036
20130621 56400 56400 54800 55900 45648
20130620 58400 58900 57500 57500 24368
20130619 59600 60000 58300 59400 20763
20130618 59400 60100 59100 59500 13749
20130617 59500 60000 59200 59400 12790
20130614 60300 60300 59300 59900 23282
20130613 62000 62000 59900 59900 39241
20130612 61000 62400 60500 62300 31751
20130611 60800 62200 60100 62100 20876
20130610 59800 61600 59800 61200 21936
20130607 60700 60700 59100 59700 22355
20130605 62400 62500 60200 60300 38522
20130604 64100 64100 62200 62700 40599
20130603 64300 64300 62800 64000 28612
20130531 64400 64700 63200 64500 39952
20130530 65800 66400 64200 64200 35535
20130529 64400 66500 63800 66500 77111
20130528 63200 64200 62600 64000 20644
20130527 65100 65600 62700 63200 85642
20130524 66300 66600 64500 65100 41502
20130523 66800 67400 6640

20120727 57500 57600 55500 57600 80128
20120726 55100 57900 55100 56500 52908
20120725 56800 56900 53600 56900 122894
20120724 54800 57000 54800 56900 73980
20120723 56300 56300 54000 55600 54024
20120720 55000 57000 54200 57000 140266
20120719 53700 55300 53700 55000 54239
20120718 56300 56500 53200 53200 57834
20120717 54400 56500 53900 56500 177165
20120716 53200 55500 52700 55000 239534
20120713 52300 53000 51800 53000 76847
20120712 53900 53900 52300 52300 89893
20120711 55000 55200 54000 54000 240519
20120710 56100 56600 54700 54900 26842
20120709 57900 57900 55500 56000 52062
20120706 58300 58700 57200 57900 29324
20120705 59100 59200 58100 58100 31047
20120704 59200 60400 59100 59200 41543
20120703 60200 60800 59500 60100 42708
20120702 61500 61500 59200 60800 37157
20120629 58300 62500 57300 61000 47844
20120628 58600 58600 57000 58500 19012
20120627 57500 58200 57200 57700 13007
20120626 57200 58500 56600 57900 24982
20120625 57200 57400 56000 57400 17122
20120622 56600 57000

...

20100208 40850 40850 39550 40450 34825
20100205 40250 40800 40000 40450 118614
20100204 41700 41850 41200 41700 50427
20100203 41250 41800 40700 41800 66680
20100202 40500 41100 40200 40800 116546
20100201 39400 40200 38600 40200 89341
20100129 39650 39850 38500 38900 112489
20100128 40250 40850 40100 40350 95478
20100127 41000 41300 40300 40300 97206
20100126 42700 42900 40400 41350 129582
20100125 43100 43100 42500 42600 56809
20100122 44000 44300 43050 43550 108653
20100121 45100 45100 44600 45000 67030
20100120 46200 46600 45150 45750 85684
20100119 46100 46450 45200 46200 130590
20100118 46800 46800 45600 46700 85951
20100115 45800 47050 45300 47050 111006
20100114 44800 46600 44800 46500 96476
20100113 45600 45600 44500 45250 90313
20100112 46050 46300 45100 45900 118620
20100111 44450 46450 44400 46100 377762
20100108 43600 44100 43050 44100 101732
20100107 43250 44500 42550 43400 145505
20100106 42950 43200 42600 43150 190524
20100105 43500 43550 42450 42950 157197
20100104 418

20081014 25794 27281 25507 27281 285237
20081013 22659 23733 22256 23733 134635
20081010 21091 21853 19972 21674 125797
20081009 21942 22659 21360 22390 114498
20081008 21629 22480 21091 21494 190525
20081007 22748 23554 21494 23196 286228
20081006 24629 24808 23286 23375 67623
20081002 25525 26152 24853 25122 152098
20081001 25793 25972 24898 25077 156418
20080930 23957 25301 23106 25166 208488
20080929 26152 26510 24763 25435 149128
20080926 23510 26465 23465 25032 593285
20080925 24360 25435 24047 24674 224659
20080924 23017 24763 22838 24674 372937
20080923 21226 23330 21226 23330 302478
20080922 23196 23465 21674 21674 236275
20080919 23420 23733 22524 22569 247546
20080918 21181 21942 20599 21718 364538
20080917 22300 23510 22032 22435 418515
20080916 21271 22032 21271 21271 285961
20080912 25614 25749 24719 24987 269913
20080911 25525 25749 24898 24942 238487
20080910 23644 25928 23644 25614 507322
20080909 25793 26465 24629 24987 508044
20080908 23733 25569 23465 25569 379166
2

20071009 70752 71738 65379 67618 312771
20071008 70036 74066 68066 71648 411714
20071005 69857 71827 66274 68066 218298
20071004 66633 69319 66095 69319 193844
20071002 66633 67886 65468 67528 350189
20071001 61438 66185 61438 63946 254064
20070928 61707 61707 58930 61259 202203
20070927 64483 64841 60632 61796 343598
20070921 58662 62692 57856 62692 260759
20070920 61349 61438 59110 59110 236505
20070919 57229 61259 55885 61259 533513
20070918 52840 53646 51497 53288 98180
20070917 54363 54632 52751 53288 70765
20070914 52034 54542 51855 54542 159059
20070913 50154 52034 49885 51766 113362
20070912 52034 53736 51497 51497 134385
20070911 53199 54094 51497 51497 156352
20070910 53199 53557 51766 53557 129423
20070907 56871 56871 55527 55527 147073
20070906 55617 56781 54990 56781 197301
20070905 58572 58662 56602 56602 116009
20070904 60453 60722 57587 57766 189075
20070903 61886 61976 59826 59826 144155
20070831 59199 60901 58841 60811 103768
20070830 60005 60363 58483 59737 85574
200

20060327 25773 26497 25735 26383 86392
20060324 25317 25926 24707 25355 86851
20060323 26078 26421 25164 25697 103855
20060322 27525 27601 25697 25697 129383
20060321 27715 28248 27487 27753 134125
20060320 28172 28172 27410 27791 65474
20060317 28324 28629 27715 28020 72985
20060316 28971 29276 28629 28781 68931
20060315 30037 30037 28781 28781 66740
20060314 28933 29352 28781 28933 114901
20060313 28933 29733 28058 28629 399354
20060310 28058 28553 27601 28058 50210
20060309 27030 28476 27030 28172 138559
20060308 26877 27715 25888 27030 69869
20060307 27715 28476 26877 26992 141564
20060306 27182 28819 26725 28210 152008
20060303 26992 27258 26002 26649 119427
20060302 26839 27410 26268 26992 169157
20060228 25888 27030 25469 26649 387112
20060227 23223 25088 23223 24898 297942
20060224 23603 23832 23032 23223 178315
20060223 23756 24289 23375 23680 182099
20060222 24517 25393 23603 23756 281491
20060221 25469 25469 24669 24746 169662
20060220 24822 26497 24593 25431 402547
20060217

20050328 6815 7766 6815 7370 41053
20050325 6487 6914 6487 6914 30642
20050324 6784 6891 6518 6601 30741
20050323 6784 7127 6776 6776 25281
20050322 7043 7195 6929 7134 27015
20050321 6784 7073 6639 7073 84697
20050318 7157 7348 6967 6967 60285
20050317 7355 7614 7233 7309 79349
20050316 7652 7804 7386 7614 90521
20050315 8642 8642 7652 7652 124748
20050314 8261 8528 7957 8375 251328
20050311 8071 8452 7728 8261 578491
20050310 7690 7995 7500 7995 245868
20050309 8109 8413 7614 7919 339955
20050308 7766 8109 7500 7919 290686
20050307 8528 8832 7842 7842 203705
20050304 8452 8528 7728 8528 364186
20050303 8642 9061 7842 8375 703232
20050302 9137 9518 8375 8908 579098
20050228 8375 8528 8071 8528 254388
20050225 7043 7614 6936 7614 521432
20050224 6091 6822 6030 6822 157386
20050223 5893 6106 5855 6091 221751
20050222 5596 6137 5596 5977 366840
20050221 5299 5535 5216 5482 179234
20050218 5040 5109 5010 5086 22774
20050217 5216 5223 5101 5109 25452
20050216 5277 5330 5185 5185 43681
2005

In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QAxContainer import *
from PyQt5.QtCore import *
import time
import pandas as pd
import sqlite3

TR_REQ_TIME_INTERVAL = 0.2

class Kiwoom(QAxWidget):
    def __init__(self):
        super().__init__()
        self._create_kiwoom_instance()
        self._set_signal_slots()

    def _create_kiwoom_instance(self):
        self.setControl("KHOPENAPI.KHOpenAPICtrl.1")

    def _set_signal_slots(self):
        self.OnEventConnect.connect(self._event_connect)
        self.OnReceiveTrData.connect(self._receive_tr_data)

    def comm_connect(self):
        self.dynamicCall("CommConnect()")
        self.login_event_loop = QEventLoop()
        self.login_event_loop.exec_()

    def _event_connect(self, err_code):
        if err_code == 0:
            print("connected")
        else:
            print("disconnected")

        self.login_event_loop.exit()

    def get_code_list_by_market(self, market):
        code_list = self.dynamicCall("GetCodeListByMarket(QString)", market)
        code_list = code_list.split(';')
        return code_list[:-1]

    def get_master_code_name(self, code):
        code_name = self.dynamicCall("GetMasterCodeName(QString)", code)
        return code_name

    def set_input_value(self, id, value):
        self.dynamicCall("SetInputValue(QString, QString)", id, value)

    def comm_rq_data(self, rqname, trcode, next, screen_no):
        self.dynamicCall("CommRqData(QString, QString, int, QString", rqname, trcode, next, screen_no)
        self.tr_event_loop = QEventLoop()
        self.tr_event_loop.exec_()

    def _comm_get_data(self, code, real_type, field_name, index, item_name):
        ret = self.dynamicCall("CommGetData(QString, QString, QString, int, QString", code,
                               real_type, field_name, index, item_name)
        return ret.strip()

    def _get_repeat_cnt(self, trcode, rqname):
        ret = self.dynamicCall("GetRepeatCnt(QString, QString)", trcode, rqname)
        return ret

    def _receive_tr_data(self, screen_no, rqname, trcode, record_name, next, unused1, unused2, unused3, unused4):
        if next == '2':
            self.remained_data = True
        else:
            self.remained_data = False

        if rqname == "opt10081_req":
            self._opt10081(rqname, trcode)

        try:
            self.tr_event_loop.exit()
        except AttributeError:
            pass

    def _opt10081(self, rqname, trcode):
        data_cnt = self._get_repeat_cnt(trcode, rqname)

        for i in range(data_cnt):
            date = self._comm_get_data(trcode, "", rqname, i, "일자")
            open = self._comm_get_data(trcode, "", rqname, i, "시가")
            high = self._comm_get_data(trcode, "", rqname, i, "고가")
            low = self._comm_get_data(trcode, "", rqname, i, "저가")
            close = self._comm_get_data(trcode, "", rqname, i, "현재가")
            volume = self._comm_get_data(trcode, "", rqname, i, "거래량")

            self.ohlcv['date'].append(date)
            self.ohlcv['open'].append(int(open))
            self.ohlcv['high'].append(int(high))
            self.ohlcv['low'].append(int(low))
            self.ohlcv['close'].append(int(close))
            self.ohlcv['volume'].append(int(volume))

if __name__ == "__main__":
    app = QApplication(sys.argv)
    kiwoom = Kiwoom()
    kiwoom.comm_connect()

    kiwoom.ohlcv = {'date': [], 'open': [], 'high': [], 'low': [], 'close': [], 'volume': []}

    # opt10081 TR 요청
    kiwoom.set_input_value("종목코드", "039490")
    kiwoom.set_input_value("기준일자", "20170224")
    kiwoom.set_input_value("수정주가구분", 1)
    kiwoom.comm_rq_data("opt10081_req", "opt10081", 0, "0101")

    while kiwoom.remained_data == True:
        time.sleep(TR_REQ_TIME_INTERVAL)
        kiwoom.set_input_value("종목코드", "039490")
        kiwoom.set_input_value("기준일자", "20170224")
        kiwoom.set_input_value("수정주가구분", 1)
        kiwoom.comm_rq_data("opt10081_req", "opt10081", 2, "0101")

    df = pd.DataFrame(kiwoom.ohlcv, columns=['open', 'high', 'low', 'close', 'volume'], index=kiwoom.ohlcv['date'])

    con = sqlite3.connect(r"C:\bigdata0214_0811\jupyterws\Algorithmic Trading\project\revision_study\data\stock.db")
    df.to_sql('039490', con, if_exists='replace')


connected
